# Procesamiento de pdfs

# columna id, partido politico,content

In [50]:
import os
import re
import pandas as pd
import pdfplumber

# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de diccionarios específicos a procesar
file_parameters = [
    {"file_name": "REVOLUCIÓN CIUDADANA - RETO _Plan de trabajo_.pdf", "exclude_pages_start": 8},
    {"file_name": "PARTIDO SOCIEDAD UNIDA MÁS ACCIÓN, SUMA _Plan de trabajo_.pdf", "exclude_pages_start": 7},
    {"file_name": "PARTIDO IZQUIERDA DEMOCRÁTICA _Plan de trabajo_.pdf","exclude_pages_start": 5},
    {"file_name": "MOVIMIENTO CENTRO DEMOCRÁTICO _Plan de trabajo_.pdf", "exclude_pages_start": 4},
    {"file_name": "MOVIMIENTO CONSTRUYE _Plan de trabajo_.pdf", "exclude_pages_start": 4},
    {"file_name": "MOVIMIENTO CREO, CREANDO OPORTUNIDADES _Plan de trabajo_.pdf", "exclude_pages_start": 4},
    {"file_name": "MOVIMIENTO AMIGO, ACCIÓN MOVILIZADORA INDEPENDIENTE GENERANDO OPORTUNIDADES _Plan de trabajo_.pdf", "exclude_pages_start": 4},
    {"file_name": "MOVIMIENTO PUEBLO IGUALDAD DEMOCRACIA _PID_ _Plan de trabajo_.pdf", "exclude_pages_start": 3},
    {"file_name": "MOVIMIENTO ACCION DEMOCRATICA NACIONAL, ADN _Plan de trabajo_.pdf", "exclude_pages_start": 3},
    {"file_name": "PARTIDO SOCIEDAD PATRIÓTICA  21 DE ENERO _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "PARTIDO UNIDAD POPULAR _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "PARTIDO SOCIALISTA ECUATORIANO _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "MOVIMIENTO DEMOCRACIA SÍ _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "PARTIDO AVANZA _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "PARTIDO SOCIAL CRISTIANO _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "MOVIMIENTO DE UNIDAD PLURINACIONAL PACHAKUTIK _Plan de trabajo_.pdf", "exclude_pages_start": 1}
]

# Función para obtener el último ID del archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    df = pd.read_csv(csv_path, sep="|", encoding="utf-8")
    if df.empty:
        return 1
    return df['ID'].iloc[-1] + 1

# Función para extraer texto del PDF excluyendo las primeras y últimas páginas
def extract_text_excluding_pages(pdf_path, exclude_pages_start, exclude_pages_end=1):
    extracted_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for i in range(exclude_pages_start, len(pdf.pages) - exclude_pages_end):
            page_text = pdf.pages[i].extract_text()
            if page_text:
                extracted_text += page_text + "\n"
    return extracted_text.strip()

# Función para limpiar el contenido del texto
def clean_content(text):

    # Eliminar viñetas comunes
    text = re.sub(r"[\u2022\u25CB\u2023\u2219\u2022\u25AA\u25B6\u25B7\u25C6\u2043\u25B8\u25BB\u2660\u25FE\u25FB]", "", text)
    text = re.sub(r'\(cid:\d+\)', '', text)
    # Eliminar enumeraciones (números seguidos de punto)
    text = re.sub(r'^\d+\.', '', text)  # Al inicio de la línea
    text = re.sub(r'\n\d+\.', '\n', text)  # En medio del texto
    
    # Reemplazar múltiples espacios con uno solo
    text = re.sub(r'\s+', ' ', text)
    
    # Eliminar espacios al inicio y final
    text = text.strip()
    
    return text

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Crear una lista para almacenar los datos
data = []

# Recorrer la lista de diccionarios específicos
for file_param in file_parameters:
    file_name = file_param["file_name"]
    exclude_pages_start = file_param["exclude_pages_start"]

    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)

    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")

        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path, exclude_pages_start=exclude_pages_start)

        # Limpiar el contenido extraído
        cleaned_content = clean_content(content)

        # Agregar los datos a la lista
        data.append([file_id, processed_name, cleaned_content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Crear un DataFrame a partir de los datos nuevos
df_new = pd.DataFrame(data, columns=['ID', 'Nombre', 'Contenido'])

# Verificar si el archivo CSV ya existe
if os.path.exists(output_csv):
    # Leer el archivo CSV existente
    df_existing = pd.read_csv(output_csv, sep="|", encoding="utf-8")
    # Concatenar los datos nuevos con los existentes
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
else:
    df_combined = df_new

# Guardar el DataFrame combinado en el archivo CSV con delimitador ";"
df_combined.to_csv(output_csv, sep=";", index=False, encoding="utf-8")

print(f"Datos agregados al archivo CSV: {output_csv}")

Datos agregados al archivo CSV: candidatos.csv


## documentos escaneados o protegidos 

In [5]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('candidatos.csv', sep=';')

# Lista de IDs que quieres buscar
ids_a_buscar = [11, 13, 15]

# Filtrar las filas donde el valor de la columna 'ID' esté en la lista
filas = df[df['ID'].isin(ids_a_buscar)]

# Mostrar las filas
print(filas)

    ID                     Nombre Contenido
10  11    PARTIDO UNIDAD POPULAR        NaN
12  13  MOVIMIENTO DEMOCRACIA SÍ        NaN
14  15  PARTIDO SOCIAL CRISTIANO        NaN


# ocr

In [2]:
import pytesseract
from pdf2image import convert_from_path
import csv
import os
csv.field_size_limit(1000000)
# Configuración global
pdf_directory = "./data/"
csv_file = "candidatos.csv"
columns = ['ID', 'Nombre', 'Contenido']

# Configurar Tesseract para Fedora
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

def procesar_pdf(ruta_pdf, id_asignado, nombre_doc):
    try:
        # Convertir PDF a imágenes
        images = convert_from_path(ruta_pdf, dpi=300)
        
        # Extraer y limpiar texto
        contenido = " ".join(
            [pytesseract.image_to_string(img, lang='spa').strip().replace('\n', ' ') 
             for img in images]
        )
        
        # Leer las filas existentes desde el CSV
        filas_existentes = []
        if os.path.exists(csv_file):
            with open(csv_file, 'r', encoding='utf-8-sig') as f:
                reader = csv.DictReader(f, delimiter=';')
                filas_existentes = list(reader)
        
        # Añadir la nueva fila con los datos del PDF
        filas_existentes.append({
            'ID': id_asignado,
            'Nombre': nombre_doc,
            'Contenido': contenido
        })
        
        # Ordenar las filas por el campo 'ID'
        filas_existentes.sort(key=lambda x: int(x['ID']))
        
        # Escribir las filas ordenadas nuevamente en el CSV
        with open(csv_file, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=columns, delimiter=';')
            writer.writeheader()
            writer.writerows(filas_existentes)
        
        return True
    except Exception as e:
        print(f"Error procesando {ruta_pdf}: {str(e)}")
        return False

# Mapeo de archivos a IDs y nombres
documentos = {
    "PARTIDO UNIDAD POPULAR _Plan de trabajo_.pdf": {"id": 11, "nombre": "PARTIDO UNIDAD POPULAR"},
    "MOVIMIENTO DEMOCRACIA SÍ _Plan de trabajo_.pdf": {"id": 13, "nombre": "MOVIMIENTO DEMOCRACIA SÍ"},
    "PARTIDO SOCIAL CRISTIANO _Plan de trabajo_.pdf": {"id": 15, "nombre": "PARTIDO SOCIAL CRISTIANO"}   
}

# Procesar todos los documentos
for archivo, datos in documentos.items():
    ruta_completa = os.path.join(pdf_directory, archivo)
    if os.path.exists(ruta_completa):
        if procesar_pdf(ruta_completa, datos['id'], datos['nombre']):
            print(f"{archivo} procesado (ID {datos['id']})")
    else:
        print(f" Archivo no encontrado: {ruta_completa}")

print("\nProceso completado.")


PARTIDO UNIDAD POPULAR _Plan de trabajo_.pdf procesado (ID 11)
MOVIMIENTO DEMOCRACIA SÍ _Plan de trabajo_.pdf procesado (ID 13)
PARTIDO SOCIAL CRISTIANO _Plan de trabajo_.pdf procesado (ID 15)

Proceso completado.


## documentos ocr

In [3]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('candidatos.csv', sep=';')

# Lista de IDs que quieres buscar
ids_a_buscar = [11,13,15]

# Filtrar las filas donde el valor de la columna 'ID' esté en la lista
filas = df[df['ID'].isin(ids_a_buscar)]

# Mostrar las filas
print(filas)

    ID                    Nombre  \
10  11    PARTIDO UNIDAD POPULAR   
12  13  MOVIMIENTO DEMOCRACIA SÍ   
13  13  MOVIMIENTO DEMOCRACIA SÍ   
15  15  PARTIDO SOCIAL CRISTIANO   
16  15  PARTIDO SOCIAL CRISTIANO   

                                            Contenido  
10  Unir al Pueblo para ser gobierno  12  Unidad P...  
12  PROGRAMA DE GOBIERNO 2025 - 2029  COMPROMISO P...  
13  PROGRAMA DE GOBIERNO 2025 - 2029  COMPROMISO P...  
15  PLAN DE TRABAJO PARTIDO SOCIAL CRISTIANO LISTA...  
16  PLAN DE TRABAJO PARTIDO SOCIAL CRISTIANO LISTA...  


# limpiar la columna content

In [4]:
import csv
import re

# Función para limpiar el contenido del texto
def clean_content(text):
    text = text.lower()
    # Eliminar viñetas comunes
    text = re.sub(r"[\u2022\u25CB\u2023\u2219\u2022\u25AA\u25B6\u25B7\u25C6\u2043\u25B8\u25BB\u2660\u25FE\u25FB]", "", text)
    
    # Eliminar (cid:...) - Referencias CID
    text = re.sub(r'\(cid:\d+\)', '', text)
    
    # Eliminar enumeraciones (números seguidos de punto)
    text = re.sub(r'^\d+\.', '', text)  # Al inicio de la línea
    text = re.sub(r'\n\d+\.', '\n', text)  # En medio del texto

    # Eliminar la enumeración de página (ejemplo: 'Página 1', 'pág. 2', etc.)
    text = re.sub(r'Página \d+', '', text)
    text = re.sub(r'pág\.\s*\d+', '', text)
    text = re.sub(r'pag\.\s*\d+', '', text)
    text = re.sub(r'Page \d+', '', text)
    text = re.sub(r'page \d+', '', text)

    # Eliminar caracteres especiales no alfabéticos ni numéricos (como @, #, $, etc.)
    text = re.sub(r'[^\w\s]', '', text)

    # Reemplazar múltiples espacios con uno solo
    text = re.sub(r'\s+', ' ', text)
    
    # Eliminar espacios al inicio y final
    text = text.strip()
    
    return text


# Leer el archivo CSV, limpiar el contenido de la columna "Contenido", y luego escribir las filas nuevamente
def limpiar_y_guardar_csv(csv_file):
    try:
        filas_existentes = []
        
        # Leer las filas existentes desde el CSV
        if os.path.exists(csv_file):
            with open(csv_file, 'r', encoding='utf-8-sig') as f:
                reader = csv.DictReader(f, delimiter=';')
                for row in reader:
                    # Limpiar el contenido de la columna "Contenido"
                    row['Contenido'] = clean_content(row['Contenido'])
                    filas_existentes.append(row)
        
        # Escribir las filas modificadas en el archivo CSV
        with open(csv_file, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=['ID', 'Nombre', 'Contenido'], delimiter=';')
            writer.writeheader()
            writer.writerows(filas_existentes)
        
        print(f"Archivo {csv_file} procesado y limpiado correctamente.")
    
    except Exception as e:
        print(f"Error procesando el archivo CSV: {str(e)}")

# Llamar a la función
limpiar_y_guardar_csv('candidatos.csv')


Archivo candidatos.csv procesado y limpiado correctamente.
